<p align="center"><img width="50%" src="https://aimodelsharecontent.s3.amazonaws.com/aimodshare_banner.jpg" /></p>


---

## Model Submission Guide: Titanic Survival Competition
Let's share our models to a centralized leaderboard, so that we can collaborate and learn from the model experimentation process...

**Instructions:**
1.   Get data in and set up X_train / X_test / y_train
2.   Preprocess data using Sklearn Column Transformer/ Write and Save Preprocessor function
3. Fit model on preprocessed data and save preprocessor function and model 
4. Generate predictions from X_test data and submit model to competition
5. Repeat submission process to improve place on leaderboard



## 1. Get data in and set up X_train, X_test, y_train objects

In [ ]:
#install aimodelshare library
! pip install aimodelshare --upgrade

In [3]:
# Get competition data
from aimodelshare import download_data
download_data('public.ecr.aws/y2e2a1d6/titanic_competition_data-repository:v1') 


Data downloaded successfully.


In [4]:
# Separate data into X_train, y_train, and X_test
import pandas as pd
training_data=pd.read_csv("titanic_competition_data/training_data.csv")
y_train= pd.get_dummies(training_data['survived'])
X_train = training_data.drop(['survived'], axis=1)

X_test=pd.read_csv("titanic_competition_data/test_data.csv")

X_train.head()

,pclass,sex,age,fare,embarked
0,3,male,28.0,7.2500,S
1,3,female,26.0,16.1000,S
2,3,female,47.0,7.0000,S
3,2,male,57.0,12.3500,Q
4,3,female,37.0,9.5875,S


##2.   Preprocess data using Sklearn Column Transformer/ Write and Save Preprocessor function


In [5]:
# In this case we use Sklearn's Column transformer in our preprocessor function

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

#Preprocess data using sklearn's Column Transformer approach

# We create the preprocessing pipelines for both numeric and categorical data.
numeric_features = ['age', 'fare']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')), #'imputer' names the step
    ('scaler', StandardScaler())])

categorical_features = ['embarked', 'sex', 'pclass']

# Replacing missing values with Modal value and then one-hot encoding.
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Final preprocessor object set up with ColumnTransformer...

preprocess = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# fit preprocessor to your data
preprocess = preprocess.fit(X_train)

In [6]:
# Here is where we actually write the preprocessor function:

# Write function to transform data with preprocessor 
# In this case we use sklearn's Column transformer in our preprocessor function

def preprocessor(data):
    preprocessed_data=preprocess.transform(data)
    return preprocessed_data

In [7]:
# check shape of X data after preprocessing it using our new function
preprocessor(X_train).shape

(1047, 10)

##3. Fit model on preprocessed data and save preprocessor function and model 


In [8]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation

model = Sequential()
model.add(Dense(64, input_dim=10, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))

model.add(Dense(2, activation='softmax')) 
                                            
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

# Fitting the NN to the Training set
model.fit(preprocessor(X_train), pd.get_dummies(y_train), 
               batch_size = 20, 
               epochs = 3, validation_split=0.25) 

Epoch 1/3
40/40 [==============================] - 1s 9ms/step - loss: 0.6698 - accuracy: 0.5847 - val_loss: 0.6196 - val_accuracy: 0.7405
Epoch 2/3
40/40 [==============================] - 0s 3ms/step - loss: 0.6135 - accuracy: 0.7083 - val_loss: 0.5781 - val_accuracy: 0.7405
Epoch 3/3
40/40 [==============================] - 0s 3ms/step - loss: 0.5840 - accuracy: 0.7236 - val_loss: 0.5500 - val_accuracy: 0.8092


#### Save preprocessor function to local "preprocessor.zip" file

In [9]:
import aimodelshare as ai
ai.export_preprocessor(preprocessor,"") 

Your preprocessor is now saved to 'preprocessor.zip'


#### Save model to local ".onnx" file

In [10]:
# Save keras model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

onnx_model = model_to_onnx(model, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

## 4. Generate predictions from X_test data and submit model to competition


In [11]:
#Set credentials using modelshare.org username/password

from aimodelshare.aws import set_credentials

#This is the unique rest api that powers this Titanic Survival Playground -- make sure to update the apiurl for new competition deployments
apiurl="https://spx5okx1qc.execute-api.us-east-1.amazonaws.com/prod/m"

set_credentials(apiurl=apiurl)

AI Modelshare Username:··········
AI Modelshare Password:··········
AI Model Share login credentials set successfully.


In [12]:
#Instantiate Competition
import aimodelshare as ai
mycompetition= ai.Competition(apiurl)

In [13]:
#Submit Model 1: 

#-- Generate predicted y values (Model 1)
#Note: Keras predict returns the predicted column index location for classification models
prediction_column_index=model.predict(preprocessor(X_test)).argmax(axis=1)

# extract correct prediction labels 
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit Model 1 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels)

Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 7

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:1128


In [14]:
# Get leaderboard to explore current best model architectures

# Get raw data in pandas data frame
data = mycompetition.get_leaderboard()

# Stylize leaderboard data
mycompetition.stylize_leaderboard(data)

,accuracy,f1_score,precision,recall,ml_framework,transfer_learning,deep_learning,model_type,depth,num_params,dense_layers,softmax_act,relu_act,loss,optimizer,model_config,username,version,memory_size
0,81.30%,79.53%,80.51%,78.90%,sklearn,False,False,LogisticRegression,nan,10.000000,nan,nan,nan,nan,liblinear,"{'C': 10, 'class_weight': None...",AIModelShare,1,nan
1,81.30%,79.53%,80.51%,78.90%,sklearn,False,False,LogisticRegression,nan,10.000000,nan,nan,nan,nan,liblinear,"{'C': 10, 'class_weight': None...",gstreett,3,nan
2,80.53%,76.64%,84.39%,75.01%,sklearn,False,False,RandomForestClassifier,nan,nan,nan,nan,nan,nan,nan,"{'bootstrap': True, 'ccp_alpha...",gstreett,5,nan
3,78.63%,76.18%,77.96%,75.33%,sklearn,False,False,GradientBoostingClassifier,nan,nan,nan,nan,nan,nan,nan,"{'ccp_alpha': 0.0, 'criterion'...",gstreett,6,nan
6,76.72%,71.62%,79.77%,70.52%,sklearn,False,False,LogisticRegression,nan,10.000000,nan,nan,nan,nan,lbfgs,"{'C': 0.01, 'class_weight': No...",AIModelShare,2,nan
7,76.72%,71.62%,79.77%,70.52%,sklearn,False,False,LogisticRegression,nan,10.000000,nan,nan,nan,nan,liblinear,"{'C': 10, 'class_weight': None...",gstreett,4,nan
8,54.96%,49.80%,50.07%,50.06%,keras,False,True,Sequential,4.000000,9154.000000,4.000000,1.000000,3.000000,str,SGD,"{'name': 'sequential', 'layers...",gstreett,7,1322832.000000


## 5. Repeat submission process to improve place on leaderboard


In [15]:
# Train and submit model 2 using same preprocessor (note that you could save a new preprocessor, but we will use the same one for this example).
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

model_2 = Sequential()
model_2.add(Dense(128, input_dim=10, activation='relu'))
model_2.add(Dropout(.3))
model_2.add(Dense(64, activation='relu'))
model_2.add(Dense(64, activation='relu'))
model_2.add(Dropout(.3))
model_2.add(Dense(64, activation='relu'))

model_2.add(Dense(2, activation='softmax')) 
                                            
# Compile model
model_2.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

# Fitting the NN to the Training set
model_2.fit(preprocessor(X_train), pd.get_dummies(y_train), 
               batch_size = 20, 
               epochs = 5, validation_split=0.25) 

Epoch 1/5
40/40 [==============================] - 1s 8ms/step - loss: 0.7012 - accuracy: 0.4586 - val_loss: 0.6810 - val_accuracy: 0.6183
Epoch 2/5
40/40 [==============================] - 0s 3ms/step - loss: 0.6703 - accuracy: 0.6166 - val_loss: 0.6560 - val_accuracy: 0.6260
Epoch 3/5
40/40 [==============================] - 0s 3ms/step - loss: 0.6548 - accuracy: 0.6204 - val_loss: 0.6372 - val_accuracy: 0.6260
Epoch 4/5
40/40 [==============================] - 0s 3ms/step - loss: 0.6444 - accuracy: 0.6306 - val_loss: 0.6204 - val_accuracy: 0.6450
Epoch 5/5
40/40 [==============================] - 0s 3ms/step - loss: 0.6307 - accuracy: 0.6382 - val_loss: 0.6031 - val_accuracy: 0.6832


In [16]:
# Save keras model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

onnx_model = model_to_onnx(model_2, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("model2.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [17]:
#Submit Model 2: 

#-- Generate predicted y values (Model 2)
#Note: Keras predict returns the predicted column index location for classification models
prediction_column_index=model_2.predict(preprocessor(X_test)).argmax(axis=1)

# extract correct prediction labels 
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit Model 2 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model2.onnx",
                                 prediction_submission=prediction_labels,
                                 preprocessor_filepath="preprocessor.zip")

Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 8

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:1128


In [18]:
# Compare two or more models
data=mycompetition.compare_models([7,8], verbose=1)
mycompetition.stylize_compare(data)

,Model_7_Layer,Model_7_Shape,Model_7_Params,Model_8_Layer,Model_8_Shape,Model_8_Params
0,Dense,"[None, 64]",704.000000,Dense,"[None, 128]",1408
1,Dense,"[None, 64]",4160.000000,Dropout,"[None, 128]",0
2,Dense,"[None, 64]",4160.000000,Dense,"[None, 64]",8256
3,Dense,"[None, 2]",130.000000,Dense,"[None, 64]",4160
4,None,None,nan,Dropout,"[None, 64]",0
5,None,None,nan,Dense,"[None, 64]",4160
6,None,None,nan,Dense,"[None, 2]",130


## Optional: Tune model within range of hyperparameters with Keras Tuner

*Simple example shown below. Consult [documentation](https://keras.io/guides/keras_tuner/getting_started/) to see full functionality.*

In [ ]:
! pip install keras_tuner

In [20]:
#Separate validation data 
from sklearn.model_selection import train_test_split
x_train_split, x_val, y_train_split, y_val = train_test_split(
     X_train, y_train, test_size=0.2, random_state=42)

In [21]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, BatchNormalization
from keras.regularizers import l1, l2, l1_l2
import keras_tuner as kt


#Define model structure & parameter search space with function
def build_model(hp):
    model = keras.Sequential()
    model.add(Dense(64, input_dim=10, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(Dense(units=hp.Int("units", min_value=32, max_value=512, step=32), #range 32-512 inclusive, minimum step between tested values is 32
                    activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(Dense(2, activation='softmax')) 
    model.compile(
        optimizer="rmsprop", loss="binary_crossentropy", metrics=["accuracy"],
    )
    return model

#initialize the tuner (which will search through parameters)
tuner = kt.RandomSearch(
    hypermodel=build_model, 
    objective="val_accuracy", # objective to optimize
    max_trials=3, #max number of trials to run during search
    executions_per_trial=3, #higher number reduces variance of results; guages model performance more accurately 
    overwrite=True,
    directory="tuning_model",
    project_name="tuning_units",
)

tuner.search(preprocessor(x_train_split), y_train_split, epochs=1, validation_data=(preprocessor(x_val), y_val))

Trial 3 Complete [00h 00m 04s]
val_accuracy: 0.7730158567428589

Best val_accuracy So Far: 0.7730158567428589
Total elapsed time: 00h 00m 13s


In [22]:
# Build model with best hyperparameters

# Get the top 2 hyperparameters.
best_hps = tuner.get_best_hyperparameters(5)
# Build the model with the best hp.
tuned_model = build_model(best_hps[0])
# Fit with the entire dataset.
tuned_model.fit(x=preprocessor(X_train), y=y_train, epochs=5)

Epoch 1/5
33/33 [==============================] - 1s 3ms/step - loss: 1.8812 - accuracy: 0.7182
Epoch 2/5
33/33 [==============================] - 0s 3ms/step - loss: 1.2622 - accuracy: 0.7832
Epoch 3/5
33/33 [==============================] - 0s 3ms/step - loss: 0.9466 - accuracy: 0.7832
Epoch 4/5
33/33 [==============================] - 0s 2ms/step - loss: 0.7710 - accuracy: 0.7784
Epoch 5/5
33/33 [==============================] - 0s 2ms/step - loss: 0.6692 - accuracy: 0.7803


In [23]:
# Save keras model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

onnx_model = model_to_onnx(tuned_model, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("tuned_model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [24]:
#Submit Model 3: 

#-- Generate predicted y values (Model 3)
prediction_column_index=tuned_model.predict(preprocessor(X_test)).argmax(axis=1)

# extract correct prediction labels 
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit to Competition Leaderboard
mycompetition.submit_model(model_filepath = "tuned_model.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels)

Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 9

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:1128


In [25]:
# Get leaderboard

data = mycompetition.get_leaderboard()
mycompetition.stylize_leaderboard(data)

,accuracy,f1_score,precision,recall,ml_framework,transfer_learning,deep_learning,model_type,depth,num_params,dense_layers,dropout_layers,softmax_act,relu_act,loss,optimizer,model_config,username,version,memory_size
0,81.30%,79.53%,80.51%,78.90%,sklearn,False,False,LogisticRegression,nan,10.000000,nan,nan,nan,nan,nan,liblinear,"{'C': 10, 'class_weight': None...",AIModelShare,1,nan
1,81.30%,79.53%,80.51%,78.90%,sklearn,False,False,LogisticRegression,nan,10.000000,nan,nan,nan,nan,nan,liblinear,"{'C': 10, 'class_weight': None...",gstreett,3,nan
2,80.53%,76.64%,84.39%,75.01%,sklearn,False,False,RandomForestClassifier,nan,nan,nan,nan,nan,nan,nan,nan,"{'bootstrap': True, 'ccp_alpha...",gstreett,5,nan
3,78.63%,76.18%,77.96%,75.33%,sklearn,False,False,GradientBoostingClassifier,nan,nan,nan,nan,nan,nan,nan,nan,"{'ccp_alpha': 0.0, 'criterion'...",gstreett,6,nan
6,76.72%,71.62%,79.77%,70.52%,sklearn,False,False,LogisticRegression,nan,10.000000,nan,nan,nan,nan,nan,lbfgs,"{'C': 0.01, 'class_weight': No...",AIModelShare,2,nan
7,76.72%,71.62%,79.77%,70.52%,sklearn,False,False,LogisticRegression,nan,10.000000,nan,nan,nan,nan,nan,liblinear,"{'C': 10, 'class_weight': None...",gstreett,4,nan
8,58.78%,55.21%,55.36%,55.16%,keras,False,True,Sequential,4.000000,17730.000000,4.000000,nan,1.000000,3.000000,str,RMSprop,"{'name': 'sequential_1', 'laye...",gstreett,9,2103472.000000
9,61.45%,43.02%,51.38%,50.32%,keras,False,True,Sequential,7.000000,18114.000000,5.000000,2.000000,1.000000,4.000000,str,SGD,"{'name': 'sequential_1', 'laye...",gstreett,8,1553904.000000
11,54.96%,49.80%,50.07%,50.06%,keras,False,True,Sequential,4.000000,9154.000000,4.000000,nan,1.000000,3.000000,str,SGD,"{'name': 'sequential', 'layers...",gstreett,7,1322832.000000


In [26]:
# Compare two or more models
data=mycompetition.compare_models([7,8,9], verbose=1)
mycompetition.stylize_compare(data)

,Model_7_Layer,Model_7_Shape,Model_7_Params,Model_8_Layer,Model_8_Shape,Model_8_Params,Model_9_Layer,Model_9_Shape,Model_9_Params
0,Dense,"[None, 64]",704.000000,Dense,"[None, 128]",1408,Dense,"[None, 64]",704.000000
1,Dense,"[None, 64]",4160.000000,Dropout,"[None, 128]",0,Dense,"[None, 64]",4160.000000
2,Dense,"[None, 64]",4160.000000,Dense,"[None, 64]",8256,Dense,"[None, 192]",12480.000000
3,Dense,"[None, 2]",130.000000,Dense,"[None, 64]",4160,Dense,"[None, 2]",386.000000
4,None,None,nan,Dropout,"[None, 64]",0,None,None,nan
5,None,None,nan,Dense,"[None, 64]",4160,None,None,nan
6,None,None,nan,Dense,"[None, 2]",130,None,None,nan
